In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Concatenate, Add, SeparableConv2D, ReLU, ZeroPadding2D, DepthwiseConv2D, TimeDistributed, LSTM, GRU, Bidirectional
from tensorflow.keras.layers import MaxPool2D, Flatten, Dense, BatchNormalization, Dropout, AveragePooling2D, Activation, GlobalAveragePooling2D
from tensorflow.keras import Model
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2


In [ ]:
pip install numpy==1.22.0

In [ ]:
pip install --upgrade numpy

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
!pip install pandas

In [ ]:
tf.test.is_built_with_cuda()
print(tf.version.VERSION)
import sys
sys.version
tf.config.list_physical_devices('GPU')
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
pip install opencv-python

In [ ]:
path = 'C:\\Users\\Mukaffi\\Desktop\\Potato'
print(os.listdir(path))

In [ ]:
dataset_path_train = os.listdir('C:\\Users\\Mukaffi\\Desktop\\Potato\\Train')
dataset_path_train

In [ ]:
dataset_path_validation = os.listdir('C:\\Users\\Mukaffi\\Desktop\\Potato\\validation')
dataset_path_validation


In [ ]:
print(dataset_path_train)

In [ ]:
print(dataset_path_validation)

In [ ]:
disease_items_train=[]

for item in dataset_path_train:
  all_disease = os.listdir('C:\\Users\\Mukaffi\\Desktop\\Potato\\Train' +'\\'+item)
  for disease in all_disease:
    disease_items_train.append((item , str('C:\\Users\\Mukaffi\\Desktop\\Potato\\Train'+'\\'+item ) +'\\'+disease ))

In [ ]:
train_disease_df = pd.DataFrame(data=disease_items_train,columns = ['Train_Disease_Type' ,'Train_Disease_Image'])
train_disease_df.head(200)

In [ ]:
disease_items_test=[]

for item in dataset_path_validation:
  all_disease = os.listdir('C:\\Users\\Mukaffi\\Desktop\\Potato\\validation' +'\\'+item)
  for disease in all_disease:
    disease_items_test.append((item , str('C:\\Users\\Mukaffi\\Desktop\\Potato\\validation'+'\\'+item ) +'\\'+disease ))

In [ ]:
test_disease_df = pd.DataFrame(data=disease_items_test,columns = ['Test_Disease_Type' ,'Test_Disease_Image'])
test_disease_df.head()

In [ ]:
img_width = 224
img_height = 224
train_images = []
train_labels = []
path = 'C:\\Users\\Mukaffi\\Desktop\\Potato\\Train\\'
for i in dataset_path_train:
  data_path = path+str(i)
  file_names = [i for i in os.listdir(data_path)]
  #print(file_names)
  for f in file_names:
    img = cv2.imread(data_path+'\\'+f)
    img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    img = cv2.resize(img,(img_width,img_height))
    train_images.append(img)
    train_labels.append(i)

In [ ]:
train_labels = np.array(train_labels)
print(train_labels)

In [ ]:
img_width = 224
img_height = 224
test_images = []
test_labels = []

path = 'C:\\Users\\Mukaffi\\Desktop\\Potato\\validation\\'
for i in dataset_path_validation:
  data_path = path+str(i)
  file_names = [i for i in os.listdir(data_path)]
  #print(file_names)
  for f in file_names:
    img = cv2.imread(data_path+'\\'+f)
    img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
    img = cv2.resize(img,(img_width,img_height))
    test_images.append(img)
    test_labels.append(i)

In [ ]:
test_labels = np.array(test_labels)
print(test_labels[:5])

In [ ]:
x_train = np.array(train_images)
x_train = x_train.astype('float32')/255.0
x_train.shape

In [ ]:
x_test = np.array(test_images)
x_test = x_test.astype('float32')/255.0
x_test.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.utils import to_categorical

y_train = train_disease_df['Train_Disease_Type'].values
#print(y_train)
y_test = test_disease_df['Test_Disease_Type'].values
#print(y_test)

In [ ]:
le =  LabelEncoder()
y_train = le.fit_transform(y_train)       #Label encoding for traditional Machine Leanring algorithms
y_train_one_hot = to_categorical(y_train) # One Hot Encoding for deep learning algorithms


le =  LabelEncoder()
y_test = le.fit_transform(y_test)         #Label encoding for traditional Machine Leanring algorithms
y_test_one_hot = to_categorical(y_test)   # One Hot Encoding for deep learning algorithms

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train_one_hot.shape)
print(y_test_one_hot.shape)

In [ ]:
x_train = np.array(x_train).reshape(2123,1,224,224, 3)
x_test = np.array(x_test).reshape(560,1,224,224, 3)

In [ ]:
print("x_train:",x_train.shape)
print("y_train:",y_train.shape)
print("x_test:",x_test.shape)
print("y_test:",y_test.shape)

# **Expansion Block**

In [ ]:
def expansion_block(x,t,filters,block_id):
    prefix = 'block_{}_'.format(block_id)
    total_filters = t*filters
    x = TimeDistributed(Conv2D(total_filters,1,padding='same',use_bias=False, name =    prefix +'expand'))(x)
    x = TimeDistributed(BatchNormalization(name=prefix +'expand_bn'))(x)
    x = TimeDistributed(ReLU(6,name = prefix +'expand_relu'))(x)
    return x

# **Depthwise Block**

In [ ]:
def depthwise_block(x,stride,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = TimeDistributed(DepthwiseConv2D(3,strides=(stride,stride),padding ='same', use_bias = False, name = prefix + 'depthwise_conv'))(x)
    x = TimeDistributed(BatchNormalization(name=prefix +'dw_bn'))(x)
    x = TimeDistributed(ReLU(6,name = prefix +'dw_relu'))(x)
    return x

# **Projection Block**

In [ ]:
def projection_block(x,out_channels,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = TimeDistributed(Conv2D(filters=out_channels,kernel_size = 1,   padding='same',use_bias=False,name= prefix + 'compress'))(x)
    x = TimeDistributed(BatchNormalization(name=prefix +'compress_bn'))(x)
    return x

In [ ]:
def Bottleneck(x,t,filters, out_channels,stride,block_id):
    y = expansion_block(x,t,filters,block_id)
    y = depthwise_block(y,stride,block_id)
    y = projection_block(y, out_channels,block_id)
    if y.shape[-1]==x.shape[-1]:
       y =tf.keras.layers.add([x,y])
    return y

In [ ]:
image_height = 224
image_width = 224
image_channels = 3
image_dimension = (1,image_height,image_width,image_channels)   #shape (height, width, channels)
input_image = Input(shape = image_dimension) # inp = layers.Input((None,128,216,1), ragged=True)
x = TimeDistributed(Conv2D(32,3,strides=(2,2),padding='same', use_bias=False))(input_image)
x = TimeDistributed(BatchNormalization(name='conv1_bn'))(x)
x = TimeDistributed(ReLU(6, name='conv1_relu'))(x)
# 17 Bottlenecks
x = depthwise_block(x,stride=1,block_id=1)
x = projection_block(x, out_channels=16,block_id=1)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 24, stride = 2,block_id = 2)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 24, stride = 1,block_id = 3)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 2,block_id = 4)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 5)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 6)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 2,block_id = 7)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 8)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 9)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 10)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 11)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 12)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 13)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 2,block_id = 14)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 15)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 16)
x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 320, stride = 1,block_id = 17)
x = TimeDistributed(Conv2D(filters = 1280,kernel_size = 1,padding='same',use_bias=False, name = 'last_conv'))(x)
x = TimeDistributed(BatchNormalization(name='last_bn'))(x)
x = TimeDistributed(ReLU(6,name='last_relu'))(x)
x = TimeDistributed(GlobalAveragePooling2D(name='global_average_pool'))(x)
x = LSTM(64,return_sequences=False)(x)   #lstm(no_of_neurons, input_shape=[batch, timesteps, feature])
output = Dense(units = 7,activation = 'softmax',name='prediction')(x)
model = Model (inputs = input_image , outputs = output , name = 'MobileNetV2_model')

In [ ]:
model.compile(optimizer='RMSprop' , loss='sparse_categorical_crossentropy',metrics =['accuracy']) 
#categorical_crossentropy (cce) produces a one-hot array containing the probable match for each category,
#sparse_categorical_crossentropy (scce) produces a category index of the most likely matching category.

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
print(tf.version.VERSION)

In [ ]:
import sys
sys.version

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
history = model.fit(x_train,y_train,epochs = 10,validation_data = (x_test, y_test),shuffle=True,batch_size=8)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
#On testing data
predictions = model.predict(x_test)
predictions

In [ ]:
#By adding the axis argument, numpy looks at the rows and columns individually.
#axis=1 means that the operation is performed across the rows of log_preds.
y_pred = np.argmax(predictions,axis=1) #from log probabilities to 0 or 1
y_pred

In [ ]:
print(len(y_pred))

In [ ]:
predicted_labels = le.inverse_transform(y_pred)
predicted_labels

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, predicted_labels)
#print(cm)
sns.heatmap(cm, annot=True)

In [ ]:
#cmap="YlGnBu"
#cmap="Blues"
#cmap="BuPu"
#cmap="Greens"

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels, predicted_labels)
#print(cm)
#fig, ax = plt.subplots(figsize=(10,10)) 
plt.figure(figsize=(8,6)) 
sns.heatmap(cm, annot=True, fmt=".0f",cmap="YlGnBu",vmin=0, linewidth = .5,vmax=560,annot_kws={"size": 10},xticklabels=["Black Scurf", "Blackleg","Common Scab", "Dry Rot", "Healthy Potatoes"," Miscellaneous"," Pink Rot"], yticklabels=["Black Scurf", "Blackleg","Common Scab", "Dry Rot", "Healthy Potatoes"," Miscellaneous"," Pink Rot"])
plt.show()

In [ ]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test,y_pred))

In [ ]:
from sklearn import metrics
print ("Precision = ", metrics.precision_score(y_test,y_pred,average='weighted', labels=np.unique(y_pred)))

In [ ]:
from sklearn import metrics
print ("Recall = ", metrics.recall_score(y_test,y_pred,average='weighted', labels=np.unique(y_pred)))

In [ ]:
from sklearn import metrics
print ("F1 Score = ", metrics.f1_score(y_test,y_pred,average='weighted', labels=np.unique(y_pred)))

In [ ]:
n1 = 0 #Select the index of image to be loaded for testing
n2 = 150
n3 = 61

img1 = x_test[n1]
img2 = x_test[n2]
img3 = x_test[n3]

input_img1 = np.expand_dims(img1, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img2 = np.expand_dims(img2, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img3 = np.expand_dims(img3, axis=0) #Expand dims so the input is (num images, x, y, c)


input_img_features_1 = model.predict(input_img1)
input_img_features_2 = model.predict(input_img2)
input_img_features_3 = model.predict(input_img3)


input_img_features_1 = np.argmax(input_img_features_1)
input_img_features_2 = np.argmax(input_img_features_2)
input_img_features_3 = np.argmax(input_img_features_3)


input_img_features_1 = input_img_features_1.reshape(-1)
input_img_features_2 = input_img_features_2.reshape(-1)
input_img_features_3 = input_img_features_3.reshape(-1)


prediction_RF_1 = le.inverse_transform(input_img_features_1)  #Reverse the label encoder to original name
prediction_RF_2 = le.inverse_transform(input_img_features_2)  #Reverse the label encoder to original name
prediction_RF_3 = le.inverse_transform(input_img_features_3)  #Reverse the label encoder to original name

prediction_RF_text1 = ' '.join(prediction_RF_1)
prediction_RF_text2 = ' '.join(prediction_RF_2)
prediction_RF_text3 = ' '.join(prediction_RF_3)

img1 = np.array(img1).reshape(224,224,3)
img2 = np.array(img2).reshape(224,224,3)
img3 = np.array(img3).reshape(224,224,3)

In [ ]:
cv2.rectangle(img1, (0, 0 - 55), (0 + 155, 30 - 5), (255,0,0), -1) #B2007F
cv2.rectangle(img1, (0, 0), (224, 224), (255,0,0), 3)  #Syntax: cv2.rectangle(image, start_point, end_point, color, thickness)
cv2.putText(img1,prediction_RF_text1, (5 , 20), cv2.FONT_HERSHEY_SIMPLEX ,0.5, (255, 255 , 255) , 1)  #Syntax: cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])


cv2.rectangle(img2, (0, 0 - 55), (0 + 155, 30 - 5), (255,0,0), -1)
cv2.rectangle(img2, (0, 0), (224, 224), (255,0,0), 3)  #Syntax: cv2.rectangle(image, start_point, end_point, color, thickness)
cv2.putText(img2,prediction_RF_text2, (5 , 20), cv2.FONT_HERSHEY_SIMPLEX ,0.5, (255, 255 , 255) , 1)  #Syntax: cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])


cv2.rectangle(img3, (0, 0 - 55), (0 + 155, 30 - 5), (255,0,0), -1)
cv2.rectangle(img3, (0, 0), (224, 224), (255,0,0), 3)  #Syntax: cv2.rectangle(image, start_point, end_point, color, thickness)
cv2.putText(img3,prediction_RF_text3, (5 , 20), cv2.FONT_HERSHEY_SIMPLEX ,0.5, (255, 255 , 255), 1)  #Syntax: cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])


plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.axis('off')
plt.imshow(img1)
plt.subplot(1, 3, 2)
plt.axis('off')
plt.imshow(img2)
plt.subplot(1, 3, 3)
plt.axis('off')
plt.imshow(img3)